In [6]:
#Importing ncessary libraries
import dask
import pystac_client
import planetary_computer as pc
import geogif
import stackstac as ss
import dask.array as da
import matplotlib.pyplot as plt

### 1. Starts a local Dask cluster.

In [2]:
#Starting a local Dask cluster
from dask.distributed import Client, LocalCluster
cluster = LocalCluster()
client = Client(cluster)
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: /user/spaudel@clarku.edu/proxy/8787/status,
Dashboard: /user/spaudel@clarku.edu/proxy/8787/status,Workers: 4
Total threads: 8,Total memory: 32.00 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:36793,Workers: 4
Dashboard: /user/spaudel@clarku.edu/proxy/8787/status,Total threads: 8
Started: Just now,Total memory: 32.00 GiB
Comm: tcp://127.0.0.1:42015,Total threads: 2
Dashboard: /user/spaudel@clarku.edu/proxy/35891/status,Memory: 8.00 GiB
Nanny: tcp://127.0.0.1:41573,


### 2. Searches the API for Sentinel-2 surface reflectance scenes between Jan 1st, 2017 and Dec 31st, 2022 across the following bbox that have a cloud cover of less than 50%:

In [3]:
#Connecting the API 
api_url='https://earth-search.aws.element84.com/v1'
catalog = pystac_client.Client.open(api_url)

#Searching Sentinel-2 surface reflectance scenes in above cateloge where...
datetime='2017-01-01/2022-12-31' # between Jan 1st,2017 and 31st, 2022
bbox = (35.16402630997399, 11.240162235950265, 35.17440912681519, 11.249823496541282)
cloud_cover=["eo:cloud_cover<50"] #Cloud cover of less than 50%

search = catalog.search(
    collections=['sentinel-2-l2a'],
    bbox=bbox,
    datetime=datetime,
    query=cloud_cover)

#Printing the number of the items from above criteria
items3=search.item_collection()
len(items3)

350

In [9]:
%%time
items = pc.sign(search)
len(items)

CPU times: user 1.05 s, sys: 0 ns, total: 1.05 s
Wall time: 1.03 s


350

In [11]:
%%time
#setting the bound box for searching the scenes
assets=['green', 'nir']
stack = stackstac.stack(items3, bounds_latlon=bbox, assets=assets)
stack

#mamba update -c conda-forge stackstac=0.5.0 to solve problem and restart the kernel

CPU times: user 114 ms, sys: 4.97 ms, total: 119 ms
Wall time: 116 ms


/srv/conda/envs/notebook/lib/python3.11/site-packages/stackstac/prepare.py:363: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  times = pd.to_datetime(


<xarray.DataArray 'stackstac-78c549d76509fa802866e848ffac6a64' (time: 350,
                                                                band: 2,
                                                                y: 109, x: 115)>
dask.array<fetch_raster_window, shape=(350, 2, 109, 115), dtype=float64, chunksize=(1, 1, 109, 115), chunktype=numpy.ndarray>
Coordinates: (12/54)
  * time                                     (time) datetime64[ns] 2017-01-05...
    id                                       (time) <U24 'S2A_36PYT_20170105_...
  * band                                     (band) <U5 'green' 'nir'
  * x                                        (x) float64 7.362e+05 ... 7.374e+05
  * y                                        (y) float64 1.244e+06 ... 1.243e+06
    view:sun_azimuth                         (time) float64 147.8 ... 149.2
    ...                                       ...
    proj:transform                           object {0, 10, 1300020, -10, 699...
    gsd                                      int64 10
    common_name                              (band) <U5 'green' 'nir'
    center_wavelength                        (band) float64 0.56 0.842
    full_width_half_max                      (band) float64 0.045 0.145
    epsg                                     int64 32636
Attributes:
    spec:        RasterSpec(epsg=32636, bounds=(736250.0, 1243400.0, 737400.0...
    crs:         epsg:32636
    transform:   | 10.00, 0.00, 736250.00|\n| 0.00,-10.00, 1244490.00|\n| 0.0...
    resolution:  10.0

### 3. Uses stackstac to retrieve bands B03 (Green) and B08 (NIR) from the scenes found in step 2.

In [15]:
# Band names are not trivial from the initial query
stack.band
# Using common_name for bands
stack = stack.assign_coords(band=stack.common_name.fillna(stack.band).rename("band"))
stack.band


<xarray.DataArray 'band' (band: 2)>
array(['green', 'nir'], dtype='<U5')
Coordinates: (12/20)
  * band                                     (band) <U5 'green' 'nir'
    s2:datatake_type                         <U8 'INS-NOBS'
    s2:saturated_defective_pixel_percentage  int64 0
    instruments                              <U3 'msi'
    proj:epsg                                int64 32636
    constellation                            <U10 'sentinel-2'
    ...                                       ...
    proj:transform                           object {0, 10, 1300020, -10, 699...
    gsd                                      int64 10
    common_name                              (band) <U5 'green' 'nir'
    center_wavelength                        (band) float64 0.56 0.842
    full_width_half_max                      (band) float64 0.045 0.145
    epsg                                     int64 32636

In [18]:
#Selecting the B03 and B08
green = stack.sel(band=["green"])
nir = stack.sel(band=["nir"])

### 4. Calculates NDWI for all pixels within the bbox for each scene from step 2.

In [21]:
#Creating a function to calculate NDWI for all pixels 
def ndwi(green, nir):
    return (green - nir) / (green + nir)

ndwi_stack = ndwi(green, nir) #Applying the function
ndwi_stack #Printing a value

<xarray.DataArray 'stackstac-78c549d76509fa802866e848ffac6a64' (time: 350,
                                                                band: 0,
                                                                y: 109, x: 115)>
dask.array<truediv, shape=(350, 0, 109, 115), dtype=float64, chunksize=(1, 0, 109, 115), chunktype=numpy.ndarray>
Coordinates: (12/54)
  * time                                     (time) datetime64[ns] 2017-01-05...
  * band                                     (band) <U5 
  * x                                        (x) float64 7.362e+05 ... 7.374e+05
  * y                                        (y) float64 1.244e+06 ... 1.243e+06
    id                                       (time) <U24 'S2A_36PYT_20170105_...
    view:sun_azimuth                         (time) float64 147.8 ... 149.2
    ...                                       ...
    proj:transform                           object {0, 10, 1300020, -10, 699...
    gsd                                      int64 10
    common_name                              (band) <U5 
    center_wavelength                        (band) float64 
    full_width_half_max                      (band) float64 
    epsg                                     int64 32636

### 5.Calculates the mean NDWI across all pixels within the bbox at each time.

In [24]:
#Calculating the mean ndwi
mean_ndwi = ndwi_stack.mean(dim=["x", "y"])
mean_ndwi

<xarray.DataArray 'stackstac-78c549d76509fa802866e848ffac6a64' (time: 350,
                                                                band: 0)>
dask.array<mean_agg-aggregate, shape=(350, 0), dtype=float64, chunksize=(1, 0), chunktype=numpy.ndarray>
Coordinates: (12/52)
  * time                                     (time) datetime64[ns] 2017-01-05...
  * band                                     (band) <U5 
    id                                       (time) <U24 'S2A_36PYT_20170105_...
    view:sun_azimuth                         (time) float64 147.8 ... 149.2
    earthsearch:boa_offset_applied           (time) bool False False ... True
    view:sun_elevation                       (time) float64 49.99 51.97 ... 49.9
    ...                                       ...
    proj:transform                           object {0, 10, 1300020, -10, 699...
    gsd                                      int64 10
    common_name                              (band) <U5 
    center_wavelength                        (band) float64 
    full_width_half_max                      (band) float64 
    epsg                                     int64 32636

### 6. Generates a figure and plots mean NDWI values from step 5 vs time. 

In [ ]:
#Generating the figure
fig,ax = plt.subplots()
ax.scatter(x=mean_ndwi.time, y=mean_ndwi)
ax.set_ylim(-1,1)